# Project Title :  Customer Segmentation Using RFM Analysis
## Objective : 
* 1.Analyze customer behavior 
* 2.classifie customers into 3 groups "High, Mid, Low" based on their purchase history using three key metrics: Recency, Frequency, and Monetary Value

# Import Libraries

In [35]:
#%pip install nbformat --upgrade

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors
from plotly.subplots import make_subplots
%matplotlib inline

In [3]:
from datetime import datetime as dt , timedelta

# Read Data 

In [4]:
data = pd.read_csv("online_retail.csv")

In [5]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


# EDA

In [7]:
data.shape

(541909, 8)

In [8]:
print(f"Data Columns :  {data.columns.tolist()}")

Data Columns :  ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [10]:
data.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900,4070,4223,NaN,23260,NaN,NaN,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114,2313,2369,NaN,1114,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,NaN,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,NaN,4.130000,16791.000000,NaN


In [11]:
print(f"Number of Duplicated Rows : {data.duplicated().sum()}")

Number of Duplicated Rows : 5268


In [12]:
data.isnull().sum() 

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Data Cleaning

In [13]:
data.isnull().sum()/len(data) * 100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [14]:
data.dropna(subset=['CustomerID'],inplace=True)

In [15]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Data Transformation

In [16]:
data['TotalAmount'] = data['Quantity'] * data['UnitPrice']

In [17]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [18]:
referrence_date = data['InvoiceDate'].max() + timedelta(days=1)

In [19]:
referrence_date

Timestamp('2011-12-10 12:50:00')

# Analysis and visualization

KPI'S

In [29]:
Number_of_unique_customers = data['CustomerID'].nunique()
print(f"Number of Unique Customers : {Number_of_unique_customers}")

Number_of_unique_products = data['StockCode'].nunique()
print(f"Number of Unique Products : {Number_of_unique_products}")

Number_of_unique_invoices = data['InvoiceNo'].nunique()
print(f"Number of Unique Invoices : {Number_of_unique_invoices}")

Number_of_countries = data['Country'].nunique()
print(f"Number of Countries : {Number_of_countries}")

Total_Revenue = data['TotalAmount'].sum()
print(f"Total Revenue : {Total_Revenue}")

Total_Quantity_Sold = data['Quantity'].sum()
print(f"Total Quantity Sold : {Total_Quantity_Sold}")

Avg_Sales_Amount_per_Transaction = data['TotalAmount'].mean()
print(f"Average Sales Amount per Transaction : {Avg_Sales_Amount_per_Transaction}")

Avg_Quantity_Sold_per_Transaction = data['Quantity'].mean()
print(f"Average Quantity Sold per Transaction : {Avg_Quantity_Sold_per_Transaction}")

Maximum_Sales_Amount_in_a_Transaction = data['TotalAmount'].max()
print(f"Maximum Sales Amount in a Transaction : {Maximum_Sales_Amount_in_a_Transaction}")

Minimum_Sales_Amount_in_a_Transaction = data['TotalAmount'].min()
print(f"Minimum Sales Amount in a Transaction : {Minimum_Sales_Amount_in_a_Transaction}")

Maximum_Quantity_Sold_in_a_Transaction = data['Quantity'].max()
print(f"Maximum Quantity Sold in a Transaction : {Maximum_Quantity_Sold_in_a_Transaction}")

Minimum_Quantity_Sold_in_a_Transaction = data['Quantity'].min()
print(f"Minimum Quantity Sold in a Transaction : {Minimum_Quantity_Sold_in_a_Transaction}")

Average_Revenue_per_Customer = data.groupby('CustomerID')['TotalAmount'].sum().mean()
print(f"Average Revenue per Customer : {Average_Revenue_per_Customer}")

Average_revenue_per_order = data.groupby('InvoiceNo')['TotalAmount'].sum().mean()
print(f"Average Revenue per Order : {Average_revenue_per_order}")

Number of Unique Customers : 4372
Number of Unique Products : 3684
Number of Unique Invoices : 22190
Number of Countries : 37
Total Revenue : 8300065.813999999
Total Quantity Sold : 4906888
Average Sales Amount per Transaction : 20.401853884555916
Average Quantity Sold per Transaction : 12.06130339774205
Maximum Sales Amount in a Transaction : 168469.6
Minimum Sales Amount in a Transaction : -168469.6
Maximum Quantity Sold in a Transaction : 80995
Minimum Quantity Sold in a Transaction : -80995
Average Revenue per Customer : 1898.4597012808783
Average Revenue per Order : 374.0453273546642


Vsualise KPI'S as Cards

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot grid (3 rows × 5 cols → 15 slots)
fig = make_subplots(rows=3, cols=4,
                    specs=[[{"type":"indicator"}]*4,
                           [{"type":"indicator"}]*4,
                           [{"type":"indicator"}]*4])

# Row 1
fig.add_trace(go.Indicator(mode="number", value=Number_of_unique_customers,
    title={"text": "Unique Customers"}), row=1, col=1)

fig.add_trace(go.Indicator(mode="number", value=Number_of_unique_products,
    title={"text": "Unique Products"}), row=1, col=2)

fig.add_trace(go.Indicator(mode="number", value=Number_of_unique_invoices,
    title={"text": "Unique Invoices"}), row=1, col=3)

fig.add_trace(go.Indicator(mode="number", value=Number_of_countries,
    title={"text": "Countries"}), row=1, col=4)

fig.add_trace(go.Indicator(mode="number", value=Total_Revenue,
    title={"text": "Total Revenue"}, number={"prefix":"$","valueformat":",.2f"}), row=2, col=1)

# Row 2
fig.add_trace(go.Indicator(mode="number", value=Total_Quantity_Sold,
    title={"text": "Total Quantity Sold"}), row=2, col=2)

fig.add_trace(go.Indicator(mode="number", value=Avg_Sales_Amount_per_Transaction,
    title={"text": "Avg Sales/Transaction"}, number={"prefix":"$","valueformat":",.2f"}), row=2, col=3)

fig.add_trace(go.Indicator(mode="number", value=Avg_Quantity_Sold_per_Transaction,
    title={"text": "Avg Qty/Transaction"}, number={"valueformat":",.2f"}), row=2, col=4)

fig.add_trace(go.Indicator(mode="number", value=Average_Revenue_per_Customer,
    title={"text": "Avg Rev/Customer"}, number={"prefix":"$","valueformat":",.2f"}), row=3, col=2)

fig.add_trace(go.Indicator(mode="number", value=Average_revenue_per_order,
    title={"text": "Avg Rev/Order"}, number={"prefix":"$","valueformat":",.2f"}), row=3, col=3)



# Layout
fig.update_layout(
    height=800, width=1200,
    title_text="📊 Business & Transaction KPIs Dashboard",
    margin=dict(l=20, r=20, t=60, b=20)
)

fig.show()


Analysis

In [34]:
# Customer Distribution over Countries
country_counts = data['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'CustomerCount']
fig = px.choropleth(country_counts, locations='Country', locationmode='country names',
                    color='CustomerCount', hover_name='Country',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title='Customer Distribution by Country')
fig.show()

C:\Users\حازممدحتعبدالعزيزعبد\AppData\Local\Temp\ipykernel_7036\2843541138.py:4: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



In [35]:
# Top 5 Countries by Customer Count
top_countries = country_counts.head(5)
fig = px.bar(top_countries, x='Country', y='CustomerCount',
             title='Top 5 Countries by Customer Count',
             labels={'CustomerCount': 'Number of Customers', 'Country': 'Country'},
             text='CustomerCount')
fig.show()

In [ ]:
# Top 3 Countries by Revenue
country_revenue = data.groupby('Country').agg({'TotalAmount': 'sum', 'InvoiceNo': 'nunique'}).reset_index()
country_revenue.columns = ['Country', 'TotalRevenue', 'OrderCount']
fig = px.bar(country_revenue.sort_values(by='TotalRevenue', ascending=False).head(3), 
             x='Country', y='TotalRevenue',
             title='Top 3 Countries by Revenue',
             labels={'TotalRevenue': 'Total Revenue', 'Country': 'Country'},
             text='TotalRevenue')
fig.show()

In [38]:
# Top 3 Countries by Orders
country_orders = data['Country'].value_counts().reset_index()
country_orders.columns = ['Country', 'OrderCount']
fig = px.bar(country_orders.head(3), x='Country', y='OrderCount',
                title='Top 3 Countries by Orders',
                labels={'OrderCount': 'Number of Orders', 'Country': 'Country'},
                text='OrderCount')
fig.show()

In [39]:
# Top 10 Products by Revenue
top_products = data.groupby('StockCode').agg({'TotalAmount': 'sum', 'Description': 'first'}).reset_index()
top_products = top_products.sort_values(by='TotalAmount', ascending=False).head(10)
fig = px.bar(top_products, x='Description', y='TotalAmount',
             title='Top 10 Products by Revenue',
             labels={'TotalAmount': 'Total Revenue', 'Description': 'Product Description'},
             text='TotalAmount')
fig.show()

In [40]:
# top 10 Products by Quantity Sold
top_products_qty = data.groupby('StockCode').agg({'Quantity': 'sum', 'Description': 'first'}).reset_index()
top_products_qty = top_products_qty.sort_values(by='Quantity', ascending=False).head(10)
fig = px.bar(top_products_qty, x='Description', y='Quantity',
             title='Top 10 Products by Quantity Sold',
                labels={'Quantity': 'Total Quantity Sold', 'Description': 'Product Description'},
                text='Quantity')
fig.show()

# RFM Model

In [54]:
RFM = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (referrence_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalAmount': 'sum'})

In [55]:
RFM.rename(columns={'InvoiceDate':'Recency',
                    'InvoiceNo':'Frequency',
                    'TotalAmount':'MonetaryValue'}, inplace=True)

In [56]:
RFM.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [57]:
quantiles = RFM.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
quantiles

{'Recency': {0.25: 17.0, 0.5: 50.0, 0.75: 143.0},
 'Frequency': {0.25: 17.0, 0.5: 42.0, 0.75: 102.0},
 'MonetaryValue': {0.25: 293.3625, 0.5: 648.075, 0.75: 1611.725}}

In [58]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4


In [59]:
RFM['R'] = RFM['Recency'].apply(RScore, args=('Recency',quantiles,))
RFM['F'] = RFM['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
RFM['M'] = RFM['MonetaryValue'].apply(FMScore, args=('MonetaryValue',quantiles,))


In [60]:
RFM.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2


In [61]:
RFM["RFM_Segment"] = RFM.R.map(str) + RFM.F.map(str) + RFM.M.map(str)
RFM["RFM_Score"] = RFM[['R','F','M']].sum(axis=1)

In [62]:
RFM.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3
12347.0,2,182,4310.00,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


In [63]:
print(RFM['RFM_Score'].max())
print(RFM['RFM_Score'].min())


12
3


In [ ]:
segmant_labels = {"low_level","Mid_level","High_level"}

def segment(x):
    if x < 5:
        return "Low_level"
    elif (x >= 5) & (x < 9):
        return "Mid_level"
    else:
        return "High_level"

In [65]:
RFM["RFM_Segment_Level"] = RFM["RFM_Score"].apply(segment)

In [66]:
RFM.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,RFM_Segment_Level
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,Low_level
12347.0,2,182,4310.00,4,4,4,444,12,High_level
12348.0,75,31,1797.24,2,2,4,224,8,Mid_level
12349.0,19,73,1757.55,3,3,4,334,10,High_level
12350.0,310,17,334.40,1,1,2,112,4,Low_level


In [67]:
segmant_counts = RFM["RFM_Segment_Level"].value_counts().reset_index()
segmant_counts.columns = ['RFM_Segmant','Counts']
segmant_counts = segmant_counts.sort_values(by='Counts',ascending=False)
segmant_counts

,RFM_Segmant,Counts
0,Mid_level,1899
1,High_level,1690
2,Low_level,783


In [70]:
fig = px.bar(segmant_counts, 
            x='RFM_Segmant', 
            y='Counts', 
            color='RFM_Segmant', 
            title='Customer distribution based on RFM Analysis',
            color_continuous_scale=px.colors.qualitative.Pastel)
fig.show()

In [71]:
RFM["RFM_Customer_Segment"] = ''

RFM.loc[RFM['RFM_Score'] >= 9, 'RFM_Customer_Segment'] = 'VIP/Loyal Customers'
RFM.loc[(RFM['RFM_Score'] >= 6) & (RFM['RFM_Score'] < 9), 'RFM_Customer_Segment'] = 'Potential Loyalist'
RFM.loc[(RFM['RFM_Score'] >= 4) & (RFM['RFM_Score'] < 6), 'RFM_Customer_Segment'] = 'At Risk Customers'
RFM.loc[(RFM['RFM_Score'] < 4) , 'RFM_Customer_Segment'] = 'Lost Customers'

In [72]:
RFM.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,RFM_Segment_Level,RFM_Customer_Segment
CustomerID,,,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3,Low_level,Lost Customers
12347.0,2,182,4310.00,4,4,4,444,12,High_level,VIP/Loyal Customers
12348.0,75,31,1797.24,2,2,4,224,8,Mid_level,Potential Loyalist
12349.0,19,73,1757.55,3,3,4,334,10,High_level,VIP/Loyal Customers
12350.0,310,17,334.40,1,1,2,112,4,Low_level,At Risk Customers


In [76]:
segmant_product_counts = RFM.groupby(['RFM_Segment_Level', 'RFM_Customer_Segment']).size().reset_index(name='Counts')
segmant_product_counts = segmant_product_counts.sort_values(by='Counts',ascending=False)

In [78]:
fig_treemap_segments = px.treemap(segmant_product_counts,
                                  path=['RFM_Segment_Level', 'RFM_Customer_Segment'],
                                    values='Counts',
                                    title='Customer Segments based on RFM Analysis',
                                    color='RFM_Segment_Level',
                                    color_continuous_scale=px.colors.sequential.Plasma_r)

fig_treemap_segments.show()

In [79]:
vip_segments = RFM[RFM['RFM_Customer_Segment'] == 'VIP/Loyal Customers']

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=vip_segments['Recency'],
    y=vip_segments['Frequency'],
    z=vip_segments['MonetaryValue'],
    mode='markers',
    marker=dict(
        size=8,
        color=vip_segments['MonetaryValue'], 
        colorscale='Viridis',   
        opacity=0.8
    )
))
fig.update_layout(
    title='3D Scatter plot of VIP/Loyal Customers',
    scene=dict(
        xaxis_title='Recency',
        yaxis_title='Frequency',
        zaxis_title='Monetary Value'
    )
)
fig.show()